# Prep dataset

In [17]:
import pandas as pd
import numpy as np
import os
import scipy
import scipy.stats

from google.colab import drive
drive.mount('/content/drive')
data_foloder = '/content/drive/My Drive/simulation/Gradient_shuffle/'
result = '/content/drive/My Drive/simulation/Output_simul/T5/word/'

baseline = pd.read_csv(data_foloder + 'simulation_HV_baseline_vb_response_deid_v3.csv')
incoh10 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_10v3.csv')
incoh20 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_20v3.csv')
incoh50 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_50v3.csv')
ineff10 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_10v3.csv')
ineff20 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_20v3.csv')
ineff50 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_50v3.csv')

baseline.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,grid,content,n_words
0,10455,"I'm a young man , an en an en- an engineer by ...",421
1,11689,Sure . I'm thirty three years old . My name is...,159
2,12376,Alright . um I live in not especially cool Spr...,468
3,12630,um So I'm currently twenty-nine . I was born a...,966
4,13493,Mhm . I'm a thirty five year old man who uh um...,134


In [2]:
os.getcwd()

'/content'

# Install lib and dependencies

In [3]:
# Use colab because: M1 keeps having issues with tf; 
# faster with TPU; all deidentified; don't need API key.
# Install transformers from hugging face

!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.6 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 


In [4]:
# T5 uses SentencePiece tokenizer, 
# which is implemented in C and is opaque to Python.
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.5 MB/s 


In [5]:
# Using t5-small model and tokenizer

from transformers import T5Tokenizer, TFT5Model

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5Model.from_pretrained('t5-small')

# tf: tensorflow
inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="tf") # Batch size 1
outputs = model(inputs, decoder_input_ids=inputs)

# The last hidden-state is the first element of the output tuple
last_hidden_states = outputs[0]  # becauses it takes advantage of the whole context

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5Model.

All the layers of TFT5Model were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.


In [6]:
inputs

<tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[8774,    6,   82, 1782,   19, 5295,    1]], dtype=int32)>

In [7]:
last_hidden_states

<tf.Tensor: shape=(1, 7, 512), dtype=float32, numpy=
array([[[ 1.43691644e-01,  1.43860221e-01, -4.79405001e-03, ...,
         -2.31729019e-02,  1.08577289e-04,  1.61265135e-01],
        [ 1.19308271e-01,  3.92194912e-02,  1.62899643e-02, ...,
         -7.39525035e-02,  1.65885955e-04,  2.14623004e-01],
        [ 1.86311349e-01,  9.69478935e-02, -5.15518859e-02, ...,
         -2.02436835e-01,  2.43982286e-04, -4.03858960e-01],
        ...,
        [-1.02484174e-01,  1.72806203e-01, -9.05705914e-02, ...,
         -1.09222971e-01,  1.99182105e-04, -9.81829464e-02],
        [ 4.98955436e-02,  3.99321705e-01, -7.35218301e-02, ...,
         -4.26821746e-02, -1.41399854e-04, -1.92345567e-02],
        [ 9.09105688e-02,  3.13064575e-01, -2.96856426e-02, ...,
         -4.46503200e-02,  3.76074604e-04,  4.69613001e-02]]],
      dtype=float32)>

In [8]:
inputs = tokenizer.encode("Hello, my dog is very cute", return_tensors="tf") # Batch size 1
inputs # tbd if we want to include '1' and '6' [spe special tokens; punctuations] in similarity calculation [ask Joao and Reno]

<tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[8774,    6,   82, 1782,   19,  182, 5295,    1]], dtype=int32)>

# Get contextualized word embeddings

In [9]:
# define a function to get contextualized embeddings
def get_last_hidden_state(sent):
  inputs = tokenizer.encode(sent, return_tensors="tf")  # Batch size 1
  outputs = model(inputs, decoder_input_ids=inputs)
  # The last hidden-state is the first element of the output tuple (c.f. Raffel et al 2020)
  last_hidden_states = outputs[0]  
  return last_hidden_states

# call the function 
# apply it to the dataframe
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  df["t5_word_embed"] = ''
  df["t5_word_embed"] = df['content'].apply(lambda x: get_last_hidden_state(x)) 
  df.to_csv(result + str(temp) + '.csv')

df.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (601 > 512). Running this sequence through the model will result in indexing errors


,grid,content,n_words,t5_word_embed
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"(((tf.Tensor(-0.013111066, shape=(), dtype=flo..."
1,11689,We have been using that opportunity to do more...,159,"(((tf.Tensor(-0.12850149, shape=(), dtype=floa..."
2,12376,Alright . um I live in not especially cool Spr...,468,"(((tf.Tensor(0.28029484, shape=(), dtype=float..."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"(((tf.Tensor(0.18845983, shape=(), dtype=float..."
4,13493,Mhm . I still get to play . And my stock inves...,134,"(((tf.Tensor(0.004609636, shape=(), dtype=floa..."


In [12]:
# double check if it's in good shape
df['t5_word_embed'][0].shape # 231 tokens, each token has 512 dimensions [for T5]

TensorShape([1, 529, 512])

# Stats and similarities functions

In [ ]:
# stats ignoring nan, apply to all LMs 
from numpy import nanmedian

import scipy
def iqr(x):
  return scipy.stats.iqr(np.array(x), nan_policy='omit')

from numpy import nanquantile
def q5(x):
    return np.nanquantile(np.array(x), 0.05)

def q95(x):
    return np.nanquantile(np.array(x), 0.95)

In [ ]:
# cosine_similarity, apply to all LMs
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# MV 5/10

In [13]:
# Average semantic similarity of each word in 5- or 10- words window

def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
# n: How many elements each
# list should have
test = ['Alex','broke','the','vase','accidentally','.','But','Kai','did','it','on','purpose','.']
divide_chunks(test,5)
chopped = list(divide_chunks(test,5))
print(chopped)

[['Alex', 'broke', 'the', 'vase', 'accidentally'], ['.', 'But', 'Kai', 'did', 'it'], ['on', 'purpose', '.']]


In [24]:
def combinations(lst): # get w1, w2 combinations
    # input: a list of <= 5 tokens
    cmb = []
    rightside = lst[:] # initialize a list
    for wid, w1 in enumerate(lst): # each token gets a chance to be w1
        rightside = lst[wid:] # dynamically chop off w1 from the rest of the list
        while rightside: # loop until the rest of the list is empty
            w2 = rightside.pop(0) # stack up w2
            if w2 != w1: # get rid of ['Alex', 'Alex']
                cmb.append([w1, w2])  
    return cmb

testing = ['Alex', 'broke', 'the', 'vase', 'accidentally']
test_result = combinations(testing)
print(test_result)

[['Alex', 'broke'], ['Alex', 'the'], ['Alex', 'vase'], ['Alex', 'accidentally'], ['broke', 'the'], ['broke', 'vase'], ['broke', 'accidentally'], ['the', 'vase'], ['the', 'accidentally'], ['vase', 'accidentally']]


In [16]:
# prep dataframe
# append columns to the embeddings df

mvs=['5', '10']
stats = ['_median', '_iqr', '_q5', '_q95'] 
for df in dfs:
    # create new empty columns
    for mv in mvs:
        for stat in stats:
            cur = 't5_word_mv' + mv + stat
            df[cur] = ''
df.head()

,grid,content,n_words,t5_word_embed,t5_word_mv5_median,t5_word_mv5_iqr,t5_word_mv5_q5,t5_word_mv5_q95,t5_word_mv10_median,t5_word_mv10_iqr,t5_word_mv10_q5,t5_word_mv10_q95
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"(((tf.Tensor(-0.013111066, shape=(), dtype=flo...",,,,,,,,
1,11689,We have been using that opportunity to do more...,159,"(((tf.Tensor(-0.12850149, shape=(), dtype=floa...",,,,,,,,
2,12376,Alright . um I live in not especially cool Spr...,468,"(((tf.Tensor(0.28029484, shape=(), dtype=float...",,,,,,,,
3,12630,"My is things are fantastic . No , I mean My uh...",966,"(((tf.Tensor(0.18845983, shape=(), dtype=float...",,,,,,,,
4,13493,Mhm . I still get to play . And my stock inves...,134,"(((tf.Tensor(0.004609636, shape=(), dtype=floa...",,,,,,,,


In [55]:
df['t5_word_embed'][0]

<tf.Tensor: shape=(1, 601, 512), dtype=float32, numpy=
array([[[-3.10553033e-02, -4.21899930e-03, -6.21268898e-02, ...,
         -9.61099789e-02,  3.67578468e-04,  1.27705187e-02],
        [ 1.78670580e-03, -5.48904948e-02, -8.62718299e-02, ...,
         -1.02402024e-01,  4.00742167e-04,  2.77379677e-02],
        [-6.48797005e-02,  1.07468605e-01, -2.83668507e-02, ...,
         -7.60212839e-02,  3.04235349e-04, -1.21393576e-01],
        ...,
        [ 8.11153501e-02,  2.59681195e-01, -1.54238269e-01, ...,
          9.70805958e-02, -2.47135235e-04, -2.73098737e-01],
        [ 1.07268795e-01,  2.65998274e-01, -1.26439348e-01, ...,
          8.88630673e-02, -2.35899861e-04, -1.69679955e-01],
        [ 6.79381862e-02,  3.10879678e-01, -1.08346254e-01, ...,
          9.01298374e-02, -2.72594480e-04, -1.66191489e-01]]],
      dtype=float32)>

In [30]:
df['t5_word_embed'][0][0]

<tf.Tensor: shape=(601, 512), dtype=float32, numpy=
array([[-3.10553033e-02, -4.21899930e-03, -6.21268898e-02, ...,
        -9.61099789e-02,  3.67578468e-04,  1.27705187e-02],
       [ 1.78670580e-03, -5.48904948e-02, -8.62718299e-02, ...,
        -1.02402024e-01,  4.00742167e-04,  2.77379677e-02],
       [-6.48797005e-02,  1.07468605e-01, -2.83668507e-02, ...,
        -7.60212839e-02,  3.04235349e-04, -1.21393576e-01],
       ...,
       [ 8.11153501e-02,  2.59681195e-01, -1.54238269e-01, ...,
         9.70805958e-02, -2.47135235e-04, -2.73098737e-01],
       [ 1.07268795e-01,  2.65998274e-01, -1.26439348e-01, ...,
         8.88630673e-02, -2.35899861e-04, -1.69679955e-01],
       [ 6.79381862e-02,  3.10879678e-01, -1.08346254e-01, ...,
         9.01298374e-02, -2.72594480e-04, -1.66191489e-01]], dtype=float32)>

In [48]:
# combinations 'alex broke the vase': 
# alex broke, alex the, alex vase; broke the, broke vase; the vase
# Parola et al is inspired by Pauselli et al (p76): 
# ''Coherence is the average similarity of each word to each of the other
# words in the list, regardless of order or proximity. ''
#w2 = word_embed[word_id+1] # adjacent neighbour only
temp_file = -1
for df in dfs:
    temp_file += 1
    print(temp_file)
    for mv in mvs:
        # print progress
        cur = 't5_word_mv' + mv
        print('current: ', cur)
        df[cur + '_similarity'] = '' # save the cosine similarities; all stats are derived from there

        # loop over each response
        for i in df.index:
            if type(df['t5_word_embed'][i]) != float: 
                # chop 1 big response sequence into 5/10-token chunks
                word_embed_chunk = list(divide_chunks(df['t5_word_embed'][i][0], int(mv))) # add [0] for T5 because it's 3Dtensor
                chunk_temp_collection = [] 
                # loop over each 5/10 chunk in the response
                for chunck_id, word_embed in enumerate(word_embed_chunk):
                    temp_collection = []
                    # calculate average similarities for that chunk (5 or 10 window)                   
                    cmbs = combinations(word_embed.numpy().tolist()) # apply function # add .numpy().tolist() for T5, change 2Dtensor to 2Darray to list of lists
                    for cmb in cmbs:
                        w1 = cmb[0]
                        w2 = cmb[1]
                        temp = cosine_similarity(w1, w2)
                        temp_collection.append(temp)
                    temp_sim = np.nanmean(temp_collection)
                    chunk_temp_collection.append(temp_sim) # incrementally append similarity mean to the list 

            # get a list of similarity means for that response, 
            # its len is the number of chunks that the response can be chopped into
            df[cur + '_similarity'][i] = chunk_temp_collection # similarity mv 5 or 10; store it for later reference/stats

            # add other stats here
            df[cur + '_median'][i] = np.nanmedian(chunk_temp_collection)
            df[cur + '_q5'][i] = q5(chunk_temp_collection)
            df[cur + '_q95'][i] = q95(chunk_temp_collection)
            df[cur + '_iqr'][i] = iqr(chunk_temp_collection)
    df.to_csv(result + str(temp_file) + '.csv')
df.head()

0
current:  t5_word_mv5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/

current:  t5_word_mv10
1
current:  t5_word_mv5
current:  t5_word_mv10
2
current:  t5_word_mv5
current:  t5_word_mv10
3
current:  t5_word_mv5
current:  t5_word_mv10
4
current:  t5_word_mv5
current:  t5_word_mv10
5
current:  t5_word_mv5
current:  t5_word_mv10
6
current:  t5_word_mv5
current:  t5_word_mv10


,grid,content,n_words,t5_word_embed,t5_word_mv5_median,t5_word_mv5_iqr,t5_word_mv5_q5,t5_word_mv5_q95,t5_word_mv10_median,t5_word_mv10_iqr,t5_word_mv10_q5,t5_word_mv10_q95,t5_word_mv5_similarity,t5_word_mv10_similarity
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"(((tf.Tensor(-0.013111066, shape=(), dtype=flo...",0.983521,0.022215,0.92873,0.993964,0.983987,0.025975,0.922136,0.99238,"[0.9129461159795479, 0.9325594959282899, 0.953...","[0.9015081499673665, 0.9358518982879386, 0.891..."
1,11689,We have been using that opportunity to do more...,159,"(((tf.Tensor(-0.12850149, shape=(), dtype=floa...",0.97669,0.026215,0.945717,0.992385,0.971987,0.028648,0.949639,0.988687,"[0.9559216633623338, 0.9770252020048533, 0.959...","[0.9553445089343123, 0.9594707990377562, 0.962..."
2,12376,Alright . um I live in not especially cool Spr...,468,"(((tf.Tensor(0.28029484, shape=(), dtype=float...",0.985309,0.01435,0.954166,0.994127,0.984431,0.010315,0.949719,0.993064,"[0.9678431990927466, 0.9680732564439222, 0.963...","[0.9530847510950087, 0.9621310579226882, 0.965..."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"(((tf.Tensor(0.18845983, shape=(), dtype=float...",0.989503,0.009156,0.963247,0.996076,0.988813,0.007007,0.95594,0.99458,"[0.9464997900998051, 0.9625409791434951, 0.975...","[0.9507475691988453, 0.9705453414827534, 0.963..."
4,13493,Mhm . I still get to play . And my stock inves...,134,"(((tf.Tensor(0.004609636, shape=(), dtype=floa...",0.971883,0.027828,0.944122,0.989696,0.968603,0.031736,0.941886,0.988454,"[0.9581117404340258, 0.9438558321789191, 0.945...","[0.926242021234885, 0.9475604029622535, 0.9627..."


In [49]:
df.columns

Index(['grid', 'content', 'n_words', 't5_word_embed', 't5_word_mv5_median',
       't5_word_mv5_iqr', 't5_word_mv5_q5', 't5_word_mv5_q95',
       't5_word_mv10_median', 't5_word_mv10_iqr', 't5_word_mv10_q5',
       't5_word_mv10_q95', 't5_word_mv5_similarity',
       't5_word_mv10_similarity'],
      dtype='object')

# K1:10

In [50]:
import ast # a module that evaluates mathematical expressions and statements

ks=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
stats = ['_median', '_iqr', '_q5', '_q95']
for df in dfs:
    # create new empty columns
    for k in ks:
        for stat in stats:
            cur = 't5_word_k' + k + stat
            df[cur] = ''
df.head()

,grid,content,n_words,t5_word_embed,t5_word_mv5_median,t5_word_mv5_iqr,t5_word_mv5_q5,t5_word_mv5_q95,t5_word_mv10_median,t5_word_mv10_iqr,...,t5_word_k8_q5,t5_word_k8_q95,t5_word_k9_median,t5_word_k9_iqr,t5_word_k9_q5,t5_word_k9_q95,t5_word_k10_median,t5_word_k10_iqr,t5_word_k10_q5,t5_word_k10_q95
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"(((tf.Tensor(-0.013111066, shape=(), dtype=flo...",0.983521,0.022215,0.92873,0.993964,0.983987,0.025975,...,,,,,,,,,,
1,11689,We have been using that opportunity to do more...,159,"(((tf.Tensor(-0.12850149, shape=(), dtype=floa...",0.97669,0.026215,0.945717,0.992385,0.971987,0.028648,...,,,,,,,,,,
2,12376,Alright . um I live in not especially cool Spr...,468,"(((tf.Tensor(0.28029484, shape=(), dtype=float...",0.985309,0.01435,0.954166,0.994127,0.984431,0.010315,...,,,,,,,,,,
3,12630,"My is things are fantastic . No , I mean My uh...",966,"(((tf.Tensor(0.18845983, shape=(), dtype=float...",0.989503,0.009156,0.963247,0.996076,0.988813,0.007007,...,,,,,,,,,,
4,13493,Mhm . I still get to play . And my stock inves...,134,"(((tf.Tensor(0.004609636, shape=(), dtype=floa...",0.971883,0.027828,0.944122,0.989696,0.968603,0.031736,...,,,,,,,,,,


In [59]:
temp_file = -1
ks=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

for df in dfs:
    temp_file += 1
    # loop through each k
    for k in ks:
        cur = 't5_word_k' + k 
        print('Coherence k ', k, 'temp_file: ', temp_file) # progress
        df[cur + '_similarity'] = ''
        # loop through each individual's response 
        for i in df.index:
            if type(df['t5_word_embed'][i]) != float:
                temp = []
                # calcuate similarity of word pairs at k inter-token distance
                for id,v in enumerate(df['t5_word_embed'][i][0].numpy().tolist()): # add [0].numpy().tolist() for T5 because it's 3Dtensor
                    w1 = v
                    try:
                        w2 = df['t5_word_embed'][i][0].numpy().tolist()[id + int(k)] # same here, for T5
                    except IndexError:
                        continue
                    sim = cosine_similarity(w1, w2)
                    temp.append(sim) # a list of similarity scores for that response

                # intermediate df, save 
                df[cur + '_similarity'][i] = temp
                df[cur + '_iqr'][i] = iqr(temp) # add other stats here
                df[cur + '_median'][i] = np.nanmedian(temp)
                df[cur + '_q5'][i] = q5(temp)
                df[cur + '_q95'][i] = q95(temp)
    df.to_csv(result + str(temp_file) + '.csv')
df.head()

Coherence k  1 temp_file:  0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to b

Coherence k  2 temp_file:  0
Coherence k  3 temp_file:  0
Coherence k  4 temp_file:  0
Coherence k  5 temp_file:  0
Coherence k  6 temp_file:  0
Coherence k  7 temp_file:  0
Coherence k  8 temp_file:  0
Coherence k  9 temp_file:  0
Coherence k  10 temp_file:  0
Coherence k  1 temp_file:  1
Coherence k  2 temp_file:  1
Coherence k  3 temp_file:  1
Coherence k  4 temp_file:  1
Coherence k  5 temp_file:  1
Coherence k  6 temp_file:  1
Coherence k  7 temp_file:  1
Coherence k  8 temp_file:  1
Coherence k  9 temp_file:  1
Coherence k  10 temp_file:  1
Coherence k  1 temp_file:  2
Coherence k  2 temp_file:  2
Coherence k  3 temp_file:  2
Coherence k  4 temp_file:  2
Coherence k  5 temp_file:  2
Coherence k  6 temp_file:  2
Coherence k  7 temp_file:  2
Coherence k  8 temp_file:  2
Coherence k  9 temp_file:  2
Coherence k  10 temp_file:  2
Coherence k  1 temp_file:  3
Coherence k  2 temp_file:  3
Coherence k  3 temp_file:  3
Coherence k  4 temp_file:  3
Coherence k  5 temp_file:  3
Coherence k

,grid,content,n_words,t5_word_embed,t5_word_mv5_median,t5_word_mv5_iqr,t5_word_mv5_q5,t5_word_mv5_q95,t5_word_mv10_median,t5_word_mv10_iqr,...,t5_word_k1_similarity,t5_word_k2_similarity,t5_word_k3_similarity,t5_word_k4_similarity,t5_word_k5_similarity,t5_word_k6_similarity,t5_word_k7_similarity,t5_word_k8_similarity,t5_word_k9_similarity,t5_word_k10_similarity
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"(((tf.Tensor(-0.013111066, shape=(), dtype=flo...",0.983521,0.022215,0.92873,0.993964,0.983987,0.025975,...,"[0.9752288972504843, 0.9017876913376451, 0.918...","[0.9365220735831384, 0.9148171911836481, 0.927...","[0.9017055233581988, 0.8717076607148434, 0.897...","[0.8712601706769589, 0.860991388867712, 0.8578...","[0.8649640268861334, 0.7779064287122658, 0.891...","[0.7916779893688211, 0.9149606897012171, 0.918...","[0.880505163160683, 0.8690401168699036, 0.8572...","[0.8658959946047612, 0.8592397750593131, 0.837...","[0.8236812611256937, 0.9145430499305158, 0.866...","[0.855447272470757, 0.855654239889291, 0.83615..."
1,11689,We have been using that opportunity to do more...,159,"(((tf.Tensor(-0.12850149, shape=(), dtype=floa...",0.97669,0.026215,0.945717,0.992385,0.971987,0.028648,...,"[0.970588009059031, 0.9647593285356912, 0.9770...","[0.938315538330997, 0.9557746253441863, 0.9781...","[0.9317779294428152, 0.9409136103660086, 0.965...","[0.9208099123065322, 0.9250184156582643, 0.941...","[0.9166374882593653, 0.9056606256927959, 0.954...","[0.901859543027049, 0.924332954190814, 0.96636...","[0.9215404340006778, 0.9258620045801808, 0.950...","[0.9126757916077988, 0.9166075146945828, 0.940...","[0.9175317094635275, 0.9206206429611196, 0.928...","[0.9280174758663627, 0.9213744616951989, 0.946..."
2,12376,Alright . um I live in not especially cool Spr...,468,"(((tf.Tensor(0.28029484, shape=(), dtype=float...",0.985309,0.01435,0.954166,0.994127,0.984431,0.010315,...,"[0.9743305124544541, 0.9428527069764653, 0.976...","[0.9825335993018985, 0.9633913105359564, 0.945...","[0.9844543614005793, 0.9628877852190034, 0.917...","[0.9690619772845491, 0.9221058201791554, 0.921...","[0.9330463427293797, 0.9091664042674411, 0.897...","[0.9344706455458556, 0.91849967189218, 0.95340...","[0.9214546549614417, 0.9163059585883869, 0.967...","[0.9532914690888877, 0.9266147688048303, 0.932...","[0.9637263554931933, 0.9591964084720885, 0.895...","[0.9525311252589946, 0.919654391602543, 0.9563..."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"(((tf.Tensor(0.18845983, shape=(), dtype=float...",0.989503,0.009156,0.963247,0.996076,0.988813,0.007007,...,"[0.992062847123335, 0.9254209040180763, 0.9957...","[0.919815675999539, 0.9304170142644171, 0.9616...","[0.9212264641748497, 0.9312571170131495, 0.967...","[0.9209230909075233, 0.9350417640598463, 0.939...","[0.9305492707391538, 0.956033448315957, 0.9327...","[0.9351797454385644, 0.9267602635527475, 0.937...","[0.9179044012156027, 0.9416139041581361, 0.944...","[0.9336760939743715, 0.9579786214765913, 0.969...","[0.9359201677242748, 0.9408995315630383, 0.974...","[0.9292820983671409, 0.9607642016342343, 0.964..."
4,13493,Mhm . I still get to play . And my stock inves...,134,"(((tf.Tensor(0.004609636, shape=(), dtype=floa...",0.971883,0.027828,0.944122,0.989696,0.968603,0.031736,...,"[0.9935880936237567, 0.9912621881243665, 0.934...","[0.9850206251491547, 0.9424541365067686, 0.957...","[0.9499501436781161, 0.9570140552429747, 0.920...","[0.9476948684659227, 0.917246786550089, 0.9004...","[0.9070843027831815, 0.8958248060574119, 0.880...","[0.9051884781528675, 0.8856620866736697, 0.900...","[0.8997299144999592, 0.9010818106132154, 0.907...","[0.911323320501742, 0.8966946238364978, 0.9040...","[0.8873903591899837, 0.9071586759552489, 0.906...","[0.899885989859575, 0.9086575643374606, 0.8880..."


In [60]:
df.columns 

Index(['grid', 'content', 'n_words', 't5_word_embed', 't5_word_mv5_median',
       't5_word_mv5_iqr', 't5_word_mv5_q5', 't5_word_mv5_q95',
       't5_word_mv10_median', 't5_word_mv10_iqr', 't5_word_mv10_q5',
       't5_word_mv10_q95', 't5_word_mv5_similarity', 't5_word_mv10_similarity',
       't5_word_k1_median', 't5_word_k1_iqr', 't5_word_k1_q5',
       't5_word_k1_q95', 't5_word_k2_median', 't5_word_k2_iqr',
       't5_word_k2_q5', 't5_word_k2_q95', 't5_word_k3_median',
       't5_word_k3_iqr', 't5_word_k3_q5', 't5_word_k3_q95',
       't5_word_k4_median', 't5_word_k4_iqr', 't5_word_k4_q5',
       't5_word_k4_q95', 't5_word_k5_median', 't5_word_k5_iqr',
       't5_word_k5_q5', 't5_word_k5_q95', 't5_word_k6_median',
       't5_word_k6_iqr', 't5_word_k6_q5', 't5_word_k6_q95',
       't5_word_k7_median', 't5_word_k7_iqr', 't5_word_k7_q5',
       't5_word_k7_q95', 't5_word_k8_median', 't5_word_k8_iqr',
       't5_word_k8_q5', 't5_word_k8_q95', 't5_word_k9_median',
       't5_word_k9_iqr

# TLC merge

In [ ]:
tlc = pd.read_csv('/Users/yancong/Desktop/4 clinical/00 Project Files/crossdx_clin.csv', index_col=0)
tlc = tlc[['grid', 'SSDvHC', 'group', 'tlc_01povspeech', 'tlc_02povcontent',	'tlc_03pressure',	'tlc_04distract',
	'tlc_05tangent', 'tlc_06derail', 'tlc_07incoh',	'tlc_08illogic',	'tlc_09clang',	'tlc_10neologism',
    	'tlc_11wordapprox',	'tlc_12circum',	'tlc_13lossgoal',	'tlc_14persev',	'tlc_15echo',	'tlc_16block',
        	'tlc_17stilt',	'tlc_18selfref', 'tlc_3f_inefficient',	'tlc_3f_incoherent',	'tlc_3f_impexpress']]
tlc.head()

,grid,SSDvHC,group,tlc_01povspeech,tlc_02povcontent,tlc_03pressure,tlc_04distract,tlc_05tangent,tlc_06derail,tlc_07incoh,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
1,10308,1.0,SSD,0,1,0,0,0,0,0,...,0,0,1,0,0,2,0,-0.406404,-0.069358,-0.018896
2,10311,0.0,HC,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.713934,-0.463481,0.197262
3,10316,1.0,SSD,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
4,10455,0.0,HC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.687947,-0.387319,-0.710348
5,10582,1.0,SSD,0,0,0,1,0,1,0,...,1,1,0,0,0,0,0,-0.059923,-0.107333,-0.458561


In [ ]:
# only keep the processed stats columns
temp = -1
for df in dfs:
    temp += 1
    df.drop(['content', 'n_words', 't5_word_embed'], axis = 1, inplace=True)
    df['grid'] = df['grid'].astype(str)
    tlc['grid'] = tlc['grid'].astype(str)
    df = df.merge(tlc, on=['grid'])
    df.to_csv(result + str(temp) + '_GT.csv')
df.head()

,grid,study,group,SSDvHC,t5_word_sum_2,t5_word_sum_3,t5_word_sum_4,t5_word_sum_5,t5_word_sum_6,t5_word_sum_7,t5_word_sum_8,t5_word_sum_9,t5_word_sum_10,t5_word_sum_mv5,t5_word_sum_mv10
0,5195,lpop,SSD,1,0.972583,0.971772,0.968676,0.968522,0.969242,0.966855,0.967668,0.968324,0.966018,4.829520,9.659039
1,6798,aces,SSD,1,0.973770,0.973053,0.970613,0.971635,0.970762,0.971595,0.971035,0.971521,0.970840,4.809522,9.501738
2,9202,aces,SSD,1,0.963587,0.962061,0.962367,0.957790,0.959843,0.959260,0.959872,0.958496,0.960442,4.672055,9.084550
3,9394,Remora,HC,0,0.980765,0.981047,0.978921,0.978727,0.979465,0.977928,0.979849,0.979064,0.978760,4.895304,9.790607
4,10308,Remora,SSD,1,0.966373,0.964298,0.960382,0.959926,0.959147,0.955821,0.955732,0.955320,0.955571,4.792469,9.584938


In [ ]:
df.columns

,grid,study,group,SSDvHC,t5_word_sum_2,t5_word_sum_3,t5_word_sum_4,t5_word_sum_5,t5_word_sum_6,t5_word_sum_7,...,tlc_12circum,tlc_13lossgoal,tlc_14persev,tlc_15echo,tlc_16block,tlc_17stilt,tlc_18selfref,tlc_3f_inefficient,tlc_3f_incoherent,tlc_3f_impexpress
0,5195,lpop,SSD,1,0.972583,0.971772,0.968676,0.968522,0.969242,0.966855,...,2,2,3,0,0,0,0,2.278186,0.675166,0.544449
1,6798,aces,SSD,1,0.973770,0.973053,0.970613,0.971635,0.970762,0.971595,...,1,1,1,0,0,0,2,2.365560,1.094802,-0.313254
2,9202,aces,SSD,1,0.963587,0.962061,0.962367,0.957790,0.959843,0.959260,...,0,0,0,0,0,0,0,-0.528266,-0.136307,0.037839
3,9394,Remora,HC,0,0.980765,0.981047,0.978921,0.978727,0.979465,0.977928,...,0,0,0,0,0,0,0,-0.141396,-0.553826,-0.859843
4,10308,Remora,SSD,1,0.966373,0.964298,0.960382,0.959926,0.959147,0.955821,...,0,0,1,0,0,2,0,-0.406404,-0.069358,-0.018896
